In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Vulnerability_Detection/Statement_Level')
os.getcwd()

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

#Train Word2Vec model

In [ ]:
from Word2VecTrain import *
from gensim.models import KeyedVectors
from helper import *
from STM_classification_manager import *
import pandas 
vector_length = 96

training_data = pandas.read_csv("Data/train.csv", encoding='utf-8')
validating_data = pandas.read_csv("Data/val.csv", encoding='utf-8')

data = pandas.concat([training_data, validating_data]) 
data = data.reset_index(drop=True) 

vectorizer = Word2VecTrain(vector_length, "Data/results/word2vec.wordvectors")
count = 0
for df_idx, row in data.iterrows():
        count += 1

        print("Collecting gadgets...", count, end="\r")
        
        pre_surrounding_context = get_context(data.at[df_idx,"surrounding_ctx_code_pred"], data.at[df_idx,"surrounding_ctx_code_succ"])
        bw_cdg_context = get_context(data.at[df_idx,"cdg_bw_slicing"], data.at[df_idx,"cdg_fw_slicing"])
        bw_ddg_context = get_context(data.at[df_idx,"ddg_bw_slicing"], data.at[df_idx,"ddg_fw_slicing"])
        operation_ctx_abstract = get_operation_context(data.at[df_idx,"operation_ctx"])
        vul_type = data.at[df_idx,"vul_type"]
        vectorizer.add_gadget([pre_surrounding_context, bw_cdg_context, bw_ddg_context, operation_ctx_abstract])
    
vectorizer.train_model()

#Representation learning

In [ ]:
from STM_classification_manager import *
from STM_blstm import *
from Result_Manager import *
from gensim.models import KeyedVectors
import pandas as pd
vector_length = 96
max_seq_length = 400
max_code_stmt_length = 20

def main():
    vectorizer =  KeyedVectors.load("Data/results/word2vec.wordvectors", mmap='r')
    blstm = STM_Train_BLSTM(vectorizer, data_file="Data/sampled_train.csv",name="word2vec", vector_length = vector_length, max_seq_length=max_seq_length, max_code_stmt_length=max_code_stmt_length, batch_size = 64)
    blstm.train(epochs=50)
    blstm = STM_Test_BLSTM(vectorizer, data_file="Data/test.csv",name="word2vec",vector_length = vector_length, max_seq_length=max_seq_length, max_code_stmt_length=max_code_stmt_length, batch_size = 64)
    predictions, targets = blstm.test()

    classification_accuracy_report(predictions, targets, 0.5)
if __name__ == '__main__':
  main()
